In [1]:
!pip install transformers accelerate bitsandbytes autotrain-advanced
!pip install -U triton==2.1.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 4.6 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.1.0
    Uninstalling triton-2.1.0:
      Successfully uninstalled triton-2.1.0
  Using cached triton-2.1.0-0-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.3 kB)
Using cached triton-2.1.0-0-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (89.2 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires triton==3.2.0; platform_system == "Linux" and platform_machine == "x86_64", but you have triton 2.1.0 which is incompatible.


In [ ]:
!pip list

Package                            Version
---------------------------------- -------------------
absl-py                            1.4.0
accelerate                         1.2.1
aiohappyeyeballs                   2.6.1
aiohttp                            3.11.14
aiosignal                          1.3.2
alabaster                          1.0.0
albucore                           0.0.21
albumentations                     1.4.23
ale-py                             0.10.2
alembic                            1.15.1
altair                             5.5.0
annotated-types                    0.7.0
anyio                              4.9.0
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
array_record                       0.7.1
arviz                              0.21.0
astropy                            7.0.1
astropy-iers-data                  0.2025.3.17.0.34.53
astunparse                         1.6.3
atpublic                           5.1
attrs               

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
# 1. 모델과 토크나이저 로드
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# 모델은 이미 NF4 양자화 되어있음!
model_path = "/content/drive/MyDrive/ProjectSummarizer/quantized_model"

# 4비트 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map={"": 0} # GPU 0번에 모델 로드 (환경에 맞게 조절)
)

tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token # 패딩 토큰 설정 (필요한 경우)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:195: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
# 2. 데이터셋 준비, 그리고 전처리

from datasets import load_dataset

dataset_path = "/content/drive/MyDrive/ProjectSummarizer/dataset/train.json"
dataset = load_dataset("json", data_files=dataset_path)


print(dataset["train"])
# "messages" feature를 "conversations"로 이름 변경
# dataset = dataset.rename_column("messages", "conversations")
# # 변경된 Dataset 확인
# print(dataset)
print(dataset["train"][0])  # 첫 번째 샘플 출력


# 전처리 함수 정의
def preprocess_function(examples):
    inputs = []
    for conversations in examples['messages']:
        inputs.append(tokenizer.apply_chat_template(conversations, tokenize=False))
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")
    labels = model_inputs["input_ids"].copy()
    model_inputs["labels"] = labels  # 'labels' 키를 model_inputs에 추가
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)
print(tokenized_dataset)
print(tokenized_dataset['train'][0])

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['messages'],
    num_rows: 73431
})
{'messages': [{'content': '다음을 요약하세요.', 'role': 'system'}, {'content': '조선 전기에 활동했던 관리로서 중종 때 청백리로 널리 알려진 우재 손중돈(1463∼1529)의 옛집이다. 언덕에 자리잡은 건물들의 배치는 사랑채와 안채가 ㅁ자형을 이루는데, 가운데의 마당을 중심으로 남쪽에는 사랑채, 나머지는 안채로 구성된다. 안채의 동북쪽에는 사당을 배치하고, 담으로 양쪽 옆면과 뒷면을 둘러 막아, 집의 앞쪽을 탁 트이게 하여 낮은 지대의 경치를 바라볼 수 있게 하였다. 보통 대문은 행랑채와 연결되지만, 이 집은 특이하게 대문이 사랑채와 연결되어 있다. 사랑채는 남자주인이 생활하면서 손님들을 맞이하는 공간으로, 대문의 왼쪽에 사랑방과 마루가 있다. 마루는 앞면이 트여있는 누마루로 ‘관가정(觀稼亭)’ 이라는 현판이 걸려있다. 대문의 오른쪽에는 온돌방, 부엌, 작은방들을 두었고 그 앞에 ㄷ자로 꺾이는 안채가 있다. 안채는 안주인이 살림을 하는 공간으로, 부엌, 안방, 큰 대청마루, 광으로 구성되어 있으며 사랑채의 사랑방과 연결이 된다. 네모기둥을 세우고 간소한 모습을 하고 있으나, 뒤쪽의 사당과 누마루는 둥근기둥을 세워 조금은 웅장한 느낌이 들게 했다. 사랑방과 누마루 주변으로는 난간을 돌렸고, 지붕은 안채와 사랑채가 한 지붕으로 이어져 있다. 관가정은 조선 중기의 남부지방 주택을 연구하는데 귀중한 자료가 되는 문화재이다.', 'role': 'user'}, {'content': '손중돈의 옛집은 특이하게 대문이 사랑채와 연결되어 있으며 마루는 앞면이 트여있는 누마루다. 보통 대문은 행랑채와 연결되지만, 이 집은 특이하게 대문이 사랑채와 연결되어 있다. 조선 전기에 활동했던 관리로서 중종 때 청백리로 널리 알려진 우재 손중돈(1463∼1529)의 옛집이다. 마루는 앞면이 트여있는 누마루로 ‘관가정(觀稼亭)’ 이라는

Map:   0%|          | 0/73431 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [22]:
# 3. lora(qlora) 모델 선언
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16, # LoRA 레이어의 rank (조정 가능)
    lora_alpha=32, # 스케일링 파라미터 (조정 가능)
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters() # 학습 가능한 파라미터 수 확인

trainable params: 6,815,744 || all params: 8,037,076,992 || trainable%: 0.0848


In [36]:
# 4. 학습 매개변수 설정
from transformers import TrainingArguments

output_dir = "/content/drive/MyDrive/ProjectSummarizer/checkpoint"
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    warmup_ratio=0.1,
    weight_decay=0.01,
    fp16=True,
    gradient_accumulation_steps=8,
    save_steps=500,  # 500 스텝마다 저장
    save_total_limit=1,
    logging_dir="./logs",
    save_strategy="steps", # 스텝 단위로 저장하도록 변경
    resume_from_checkpoint=True, # 저장된 체크포인트가 있으면 읽어서 다시 시작
    push_to_hub=False,
    report_to="none", # wandb의 영향력 제거
)

In [18]:
torch.cuda.empty_cache()

In [37]:
from transformers import Trainer
import torch

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset['train'],
    args=training_args,
    data_collator=lambda data: {
        'input_ids': torch.tensor([f['input_ids'] for f in data]),
        'attention_mask': torch.tensor([f['attention_mask'] for f in data]),
        'labels': torch.tensor([f['labels'] for f in data])
    }
)

trainer.train()

# 학습된 모델 저장
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Step,Training Loss
500,12.698700
1000,10.221500
1500,10.001000
2000,9.812000
2500,9.697600
3000,9.657700


KeyboardInterrupt: 

In [20]:
# 6. 추론전 결합
from peft import PeftModel

qlora_path = "/content/drive/MyDrive/ProjectSummarizer/checkpoint/checkpoint-3000"  # QLoRA 학습 모델 경로
model = PeftModel.from_pretrained(model, qlora_path)

# eval도 해야하나?

In [38]:
# 7. 추론
PROMPT = '''다음을 요약하세요.'''
instruction = '''네, 산청 산불 지휘 본부에 나와 있습니다.
헬기 진화는 해가 지면서 중단됐고요.
지금은 지상 인력 약 1천 명이 투입돼 밤샘 진화 작업에 들어갔습니다.
오후 한때 빗방울이 잠시 떨어지기도 했지만, 금방 그쳤습니다.
경남 산청과 하동 전체 진화율은 오후 6시 기준 99%까지 올랐습니다.
오늘은 아침부터 기상 상황이 좋아 일찌감치 헬기 진화가 시작됐습니다.
진화 헬기 49대가 주불이 있는 지리산 내원계곡 일대에 집중 투입돼 진화 작업을 벌였습니다.
진화대와 소방, 경찰 군인 등 약 1천6백 명과 살수차와 동물방역기를 비롯한 각종 장비 2백여 대가 투입됐습니다.
한때 천왕봉 4.5km까지 근접했던 산불을 지난밤 밤샘 진화 작업을 벌여 내원계곡 쪽으로 약 2km 후퇴시켰고, 오늘 오후엔 진화율을 99%까지 끌어올렸습니다.
이제 남은 불의 길이는 4백 미텁니다.
다만 지리산은 숲이 우거져 헬기로 물을 뿌려도 지표면에 잘 닿지 않고, 계곡에 쌓인 낙엽은 깊이가 1m가 넘습니다.
위에서 물을 뿌려 불을 끄기도 쉽지 않고 낙엽층 속에 숨어있던 불은 번번이 되살아납니다.
지리산국립공원의 산불 영향구역은 이미 132ha에 이릅니다.
진화율 99%까지 왔지만 아직 긴장을 늦출 수 없는 이유입니다.
하동 산불지역에선 뒷불 감시와 잔불 정리를 계속 하고 있습니다.
오늘 아침 한때 바람 방향이 바뀌면서 산청군 삼장면 신촌마을 등 5개 마을에 다시 주민 대피령이 내려지기도 했습니다.
산청과 하동 주민 460여 명은 아직 대피소에 머물고 있는 상태입니다.
지금까지 산청 산불 현장에서 MBC뉴스 이재경입니다.'''

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"}
]

# 메시지를 템플릿 형식으로 적용
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# 종료 조건 정의
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

# 모델을 통해 텍스트 생성
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs['input_ids'],  # input_ids 전달
        attention_mask=inputs.get('attention_mask', None),  # attention_mask가 있을 경우 전달
        max_new_tokens=1024,            # 생성할 최대 토큰 수
        eos_token_id=tokenizer.eos_token_id,  # 종료 토큰 설정
        do_sample=True,                 # 샘플링 방식 사용
        temperature=0.6,                # 창의성 조절
        top_p=0.9,                      # 확률 분포 상위 p 비율에 해당하는 후보만 선택
        pad_token_id=tokenizer.pad_token_id  # 패딩 토큰 ID (필요시 설정)
    )

# 생성된 텍스트 디코딩
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print('------')
# 출력 결과에서 프롬프트 부분 제외하고 결과 출력
print(generated_text)
print('------')

------
system

다음을 요약하세요.user

네, 산청 산불 지휘 본부에 나와 있습니다.
헬기 진화는 해가 지면서 중단됐고요.
지금은 지상 인력 약 1천 명이 투입돼 밤샘 진화 작업에 들어갔습니다.
오후 한때 빗방울이 잠시 떨어지기도 했지만, 금방 그쳤습니다.
경남 산청과 하동 전체 진화율은 오후 6시 기준 99%까지 올랐습니다.
오늘은 아침부터 기상 상황이 좋아 일찌감치 헬기 진화가 시작됐습니다.
진화 헬기 49대가 주불이 있는 지리산 내원계곡 일대에 집중 투입돼 진화 작업을 벌였습니다.
진화대와 소방, 경찰 군인 등 약 1천6백 명과 살수차와 동물방역기를 비롯한 각종 장비 2백여 대가 투입됐습니다.
한때 천왕봉 4.5km까지 근접했던 산불을 지난밤 밤샘 진화 작업을 벌여 내원계곡 쪽으로 약 2km 후퇴시켰고, 오늘 오후엔 진화율을 99%까지 끌어올렸습니다.
이제 남은 불의 길이는 4백 미텁니다.
다만 지리산은 숲이 우거져 헬기로 물을 뿌려도 지표면에 잘 닿지 않고, 계곡에 쌓인 낙엽은 깊이가 1m가 넘습니다.
위에서 물을 뿌려 불을 끄기도 쉽지 않고 낙엽층 속에 숨어있던 불은 번번이 되살아납니다.
지리산국립공원의 산불 영향구역은 이미 132ha에 이릅니다.
진화율 99%까지 왔지만 아직 긴장을 늦출 수 없는 이유입니다.
하동 산불지역에선 뒷불 감시와 잔불 정리를 계속 하고 있습니다.
오늘 아침 한때 바람 방향이 바뀌면서 산청군 삼장면 신촌마을 등 5개 마을에 다시 주민 대피령이 내려지기도 했습니다.
산청과 하동 주민 460여 명은 아직 대피소에 머물고 있는 상태입니다.
지금까지 산청 산불 현장에서 MBC뉴스 이재경입니다.assistant

지리산국립공원의 산불 영향구역은 132ha에 이르렀고 진화율이 99%까지 왔지만 아직 긴장을 늦출 수 없는 이유로 하동 산불지역에선 잔불 정리를 계속하고 있다. 지리산국립공원의 산불 영향구역은 이미 132ha에 이릅니다. 진화율 99%까지 왔지만 아직 긴장을 늦출 수 없는 이유입니다. 하동 산불지역에선 뒷불 감

In [39]:
while True:
    # 사용자로부터 입력 받기
    instruction = input("입력하실 내용을 입력하세요 (종료하려면 'exit' 입력): ")

    if instruction.lower() == 'exit':
        print("프로그램을 종료합니다.")
        break  # 'exit'을 입력하면 반복문 종료

    # 메시지를 템플릿 형식으로 적용
    messages = [
        {"role": "system", "content": f"{PROMPT}"},
        {"role": "user", "content": f"{instruction}"}
    ]

    # 메시지를 템플릿 형식으로 적용
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # 종료 조건 정의
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    # inputs를 device로 이동
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

    # 모델을 통해 텍스트 생성
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs['input_ids'],  # input_ids 전달
            attention_mask=inputs.get('attention_mask', None),  # attention_mask가 있을 경우 전달
            max_new_tokens=1024,            # 생성할 최대 토큰 수
            eos_token_id=tokenizer.eos_token_id,  # 종료 토큰 설정
            do_sample=True,                 # 샘플링 방식 사용
            temperature=0.6,                # 창의성 조절
            top_p=0.9,                      # 확률 분포 상위 p 비율에 해당하는 후보만 선택
            pad_token_id=tokenizer.pad_token_id  # 패딩 토큰 ID (필요시 설정)
        )

    # 생성된 텍스트 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 출력 결과에서 프롬프트 부분 제외하고 결과 출력
    print('------')
    print(generated_text)
    print('------')


입력하실 내용을 입력하세요 (종료하려면 'exit' 입력): 1. 탄핵소추사유는 그 대상 사실을 다른 사실과 명백하게 구분할 수 있을 정도의 구체적 사실이 기재되면 충분하다. 이 사건 소추의결서의 헌법 위배행위 부분은 소추사유가 분명하게 유형별로 구분되지 않은 측면이 있지만, 소추사유로 기재된 사실관계는 법률 위배행위 부분과 함께 보면 다른 소추사유와 명백하게 구분할 수 있을 정도로 충분히 구체적으로 기재되어 있다. 2. 가. 국회의 의사절차에 헌법이나 법률을 명백히 위반한 흠이 있는 경우가 아니면 국회 의사절차의 자율권은 권력분립의 원칙상 존중되어야 하고, 국회법 제130조 제1항은 탄핵소추의 발의가 있을 때 그 사유 등에 대한 조사 여부를 국회의 재량으로 규정하고 있으므로, 국회가 탄핵소추사유에 대하여 별도의 조사를 하지 않았다거나 국정조사결과나 특별검사의 수사결과를 기다리지 않고 탄핵소추안을 의결하였다고 하여 그 의결이 헌법이나 법률을 위반한 것이라고 볼 수 없다. 나. 국회법에 탄핵소추안에 대하여 표결 전에 반드시 토론을 거쳐야 한다는 명문 규정은 없다. 또 이 사건 소추의결 당시 토론을 희망한 의원이 없었기 때문에 탄핵소추안에 대한 제안 설명만 듣고 토론 없이 표결이 이루어졌을 뿐, 의장이 토론을 희망하는 의원이 있었는데도 토론을 못하게 하거나 방해한 사실은 없다. 다. 탄핵소추안을 각 소추사유별로 나누어 발의할 것인지, 아니면 여러 소추사유를 포함하여 하나의 안으로 발의할 것인지는 소추안을 발의하는 의원들의 자유로운 의사에 달린 것이고, 표결방법에 관한 어떠한 명문규정도 없다. 라. 탄핵소추절차는 국회와 대통령이라는 헌법기관 사이의 문제이고, 국회의 탄핵소추의결에 따라 사인으로서 대통령 개인의 기본권이 침해되는 것이 아니다. 국가기관이 국민에 대하여 공권력을 행사할 때 준수하여야 하는 법원칙으로 형성된 적법절차의 원칙을 국가기관에 대하여 헌법을 수호하고자 하는 탄핵소추절차에 직접 적용할 수 없다. 3. 헌법재판은 9인의 재판관으로 구성된 재판부에 의

KeyboardInterrupt: Interrupted by user

In [ ]:
!autotrain llm -help

usage: autotrain <command> [<args>] llm [-h] [--train] [--deploy] [--inference]
                                        [--backend BACKEND] [--model MODEL]
                                        [--project-name PROJECT_NAME] [--data-path DATA_PATH]
                                        [--train-split TRAIN_SPLIT] [--valid-split VALID_SPLIT]
                                        [--add-eos-token] [--model-max-length MODEL_MAX_LENGTH]
                                        [--padding PADDING] [--trainer TRAINER]
                                        [--use-flash-attention-2] [--log LOG]
                                        [--disable-gradient-checkpointing]
                                        [--logging-steps LOGGING_STEPS]
                                        [--eval-strategy EVAL_STRATEGY]
                                        [--save-total-limit SAVE_TOTAL_LIMIT]
                                        [--auto-find-batch-size]
                                      

In [ ]:
# --gradient-accumulation 4 \

base_model = '/content/drive/MyDrive/ProjectSummarizer/quantized_model'
finetuned_model = 'qlora-finetuned'
data_path = '/content/drive/MyDrive/ProjectSummarizer/dataset/'
checkpoint_path = "/content/drive/MyDrive/ProjectSummarizer/checkpoint"

!autotrain llm \
--train \
--model {base_model} \
--project-name {finetuned_model} \
--text-column "messages" \
--data-path {data_path} \
--lr 2e-4 \
--batch-size 2 \
--epochs 1 \
--block-size 1024 \
--warmup-ratio 0.1 \
--lora-r 16 \
--lora-alpha 32 \
--lora-dropout 0.05 \
--weight-decay 0.01 \
--mixed-precision fp16 \
--gradient-accumulation 8 \
--peft \
--quantization int4 \
--trainer sft \
--save-total-limit 1

INFO     | 2025-03-26 14:24:52 | autotrain.cli.run_llm:run:136 - Running LLM
WARNING  | 2025-03-26 14:24:52 | autotrain.trainers.common:__init__:286 - Parameters supplied but not used: train, func, inference, backend, version, config, deploy
INFO     | 2025-03-26 14:24:52 | autotrain.backends.local:create:20 - Starting local training...
INFO     | 2025-03-26 14:24:52 | autotrain.commands:launch_command:514 - ['accelerate', 'launch', '--num_machines', '1', '--num_processes', '1', '--mixed_precision', 'fp16', '-m', 'autotrain.trainers.clm', '--training_config', 'qlora-finetuned/training_params.json']
INFO     | 2025-03-26 14:24:52 | autotrain.commands:launch_command:515 - {'model': '/content/drive/MyDrive/ProjectSummarizer/quantized_model', 'project_name': 'qlora-finetuned', 'data_path': '/content/drive/MyDrive/ProjectSummarizer/dataset/', 'train_split': 'train', 'valid_split': None, 'add_eos_token': True, 'block_size': 1024, 'model_max_length': 2048, 'padding': 'right', 'trainer': 'sft'

In [ ]:
# 토크나이저 작동 방식
def count_sentences_and_tokens(text, tokenizer):
    import re

    # 문장 분리 (간단한 문장 분리기)
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())

    # 각 문장의 토큰 수 계산
    sentence_token_counts = [len(tokenizer.tokenize(sentence)) for sentence in sentences]

    return len(sentences), sentence_token_counts

# 테스트할 텍스트
text = "세계 스마트폰 생산량 중 중국에서 생산하는 비중이 지난해 70% 밑으로 떨어졌다.   스마트폰업체들이 생산 기지를 중국에서 다른 지역으로 옮기고 있기 때문이다.   더욱이 신종 코로나바이러스 감염증(코로나19) 사태로 ‘중국 공급망 리스크’가 부각되면서 스마트폰업체들의 탈(脫)중국 행렬이 거세질 것으로 보인다.      중국은…임금 상승, 미·중 분쟁, 코로나까지   25일 시장조사업체인 카운터포인트리서치에 따르면, 지난해 전 세계 스마트폰 생산량 중 중국에서 생산된 비중은 전년(72%)보다 줄어든 68%인 것으로 나타났다.   2016년에는 75%, 이듬해에는 74%였다.   카운터포인트리서치는 \"중국이 세계 스마트폰 공장으로서의 매력을 잃고 있다\"고 분석했다.   이유는 크게 세 가지다.   중국 근로자의 인건비 상승, 미·중 무역분쟁 여파, 그리고 인도 등 스마트폰 판매량이 급증하고 있는 신흥시장의 부상이다.   중국 대체할 생산기지로 인도·베트남 급부상  카운터포인트리서치는 \"특히 인도가 '메이크 인 인디아(make in IIndia)' 정책을 펴며 스마트폰업체를 유인하고 있다\"며 \"세계에서 두 번째 큰 시장으로 성장한 인도의 정책에 많은 업체가 동요하고 있다\"고 전했다.   인도는 '메이크 인 인디아'라는 자국 산업 육성정책에 따라 전기·전자 부품 관련 수입 관세를 급격히 높였다.   무관세였던 휴대전화 완제품과 부품을 인도로 들여가려면 15~20%의 관세를 내야 한다.   인도에서 팔 스마트폰은 인도에서 만들라는 메시지다.   또 베트남도 중국 대비 저렴한 인건비와 각종 개혁개방 정책을 무기로 스마트폰 공장 유치에 적극적이다.     삼성전자는 완전 철수, 애플도 인도 이전 추진 중   탈(脫) 중국의 대표적인 업체가 삼성전자다.   삼성전자는 2018년 톈진 공장을, 지난해 9월에는 후이저우 공장을 폐쇄했다.   적어도 스마트폰의 생산공장만큼은 중국에서 완전히 철수한 셈이다.   인건비 상승과 중국 내수 시장 점유율 급락이 주된 이유다.   스마트폰 업계에 따르면 삼성전자 중국 공장의 근로자 월평균 급여가 2008년 274달러(약 34만원)에서 2018년에는 832달러(104만원)로 3배 증가했다.   또 삼성전자의 중국 스마트폰 시장 점유율은 2013년 19.  7%에서 최근엔 1% 밑으로 떨어졌다.   삼성전자는 이제 스마트폰을 베트남 박닌·타이응우엔과 인도 노이다 등에서 생산하고 있다.   아이폰의 90% 이상을 위탁 생산하는 폭스콘과 페가트론도 일부 생산 라인을 인도 등으로 이전했다. "

num_sentences, token_counts = count_sentences_and_tokens(text, pipeline.tokenizer)

print(f"총 문장 수: {num_sentences}")
print(f"각 문장의 토큰 수: {token_counts}")
print(sum(token_counts))

총 문장 수: 22
각 문장의 토큰 수: [26, 22, 56, 73, 16, 32, 9, 46, 89, 39, 36, 18, 37, 37, 31, 24, 21, 58, 20, 14, 31, 32]
767


In [ ]:
import transformers
import torch

# 모델이 저장된 경로
model_path = "/content/drive/MyDrive/ProjectSummarizer/quantized_model"

# 텍스트 생성 파이프라인 초기화
pipeline = transformers.pipeline(
    "text-generation",
    model=model_path,  # 모델 경로를 지정
    model_kwargs={
        "device_map": "auto",  # GPU 또는 CPU 자동 매핑
    }
)

pipeline.model.eval()

PROMPT = '''당신은 인간과 대화하는 친절한 챗봇입니다. 질문에 대한 정보를 상황에 맞게 자세히 제공합니다. 당신이 질문에 대한 답을 모른다면, 사실은 모른다고 말합니다.'''
instruction = "복잡도 이론에서 PH는 무엇인가요?"

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"}
]
# messages = [
#     {"role": "system", "content": f"{PROMPT}"},
#     {"role": "user", "content": f"{instruction}"}
# ]


# 메시지를 템플릿 형식으로 적용
prompt = pipeline.tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# 종료 조건 정의
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# 텍스트 생성
outputs = pipeline(
    prompt,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

# 출력 결과에서 프롬프트 부분 제외하고 결과 출력
print(outputs[0]["generated_text"][len(prompt):])


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
# 사용자로부터 instruction을 반복적으로 입력받아 모델에 전달
print("반복 질문을 시작합니다. 'exit'를 입력하면 종료됩니다.")

while True:
    # 사용자로부터 질문 입력받기
    instruction = input("Enter your instruction: ")

    # 'exit'를 입력하면 종료
    if instruction.lower() == "exit":
        print("종료합니다. 감사합니다!")
        break

    # 메시지 구성
    messages = [
        {"role": "system", "content": '''당신은 인간과 대화하는 친절한 챗봇입니다. 질문에 대한 정보를 상황에 맞게 자세히 제공합니다. 당신이 질문에 대한 답을 모른다면, 사실은 모른다고 말합니다.'''},
        {"role": "user", "content": instruction}
    ]

    # 프롬프트 생성
    prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # 텍스트 생성
    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=2048,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )

    # 출력 결과
    generated_text = outputs[0]["generated_text"][len(prompt):]
    print("\nGenerated Response:")
    print(generated_text)
    print("-" * 50)  # 출력 구분선


반복 질문을 시작합니다. 'exit'를 입력하면 종료됩니다.
Enter your instruction: 네 이름은 뭐지

Generated Response:
네, 저는 "챗봇"입니다.
--------------------------------------------------
